# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/11/11/ai-live-event/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/'},
  {'type': 'linkedin page', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter page', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [10]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 9 relevant links


{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'projects page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'projects page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'external company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [11]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 4 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [12]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [13]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 7 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
microsoft/VibeVoice-Realtime-0.5B
Updated
3 days ago
•
106k
•
755
Tongyi-MAI/Z-Image-Turbo
Updated
4 days ago
•
257k
•
2.57k
zai-org/GLM-4.6V-Flash
Updated
3 days ago
•
33.2k
•
367
mistralai/Devstral-Small-2-24B-Instruct-2512
Updated
about 3 hours ago
•
10.4k
•
279
zai-org/GLM-4.6V
Updated
3 days ago
•
2.52k
•
277
Browse 1M+ models
Spaces
Running
on
Zero
MCP
Featured
1.32k
Z Image Turbo
🏃
1.32k
Generate images from text prompts
Running
on
Zero
487
Z Image Turbo
🖼
487
Generate stunning images from text prompts
Running
on
Zero
Featured


In [14]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmicrosoft/VibeVoice-Realtime-0.5B\nUpdated\n3 days ago\n•\n106k\n•\n755\nTongyi-MAI/Z-Image-Turbo\nUpdated\n4 days ago\n•\n257k\n•\n2.57k\nzai-org/GLM-4.6V-Flash\nUpdated\n3 days ago\n•\n33.2k\n•\n367\nmistralai/Devstral-Small-2-24B-Instruct-2512\nUpdated\nabout 3 hours ago\n•\n10.4k\n•\n279\nzai-org/GLM-4.6V\nUpdated\n3 days ago\n•\n2.52k\n•\n277\nBrowse 1M+ models\nSpaces\nRunning\non\nZe

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    print(result)
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links
# Hugging Face: The AI Community Building the Future

---

## About Hugging Face

Hugging Face is a vibrant AI community and the leading platform where machine learning developers, researchers, and organizations collaborate on models, datasets, and AI applications. The company fosters open collaboration to accelerate innovation across multiple AI modalities including text, image, video, audio, and even 3D.

With over **1 million models** and **250,000+ datasets** available, Hugging Face is the home base for machine learning enthusiasts and experts worldwide to create, share, and build cutting-edge AI solutions.

---

## Platform Highlights

- **Collaboration Hub:** Host and collaborate on unlimited public models, datasets, and applications.
- **Explore AI Apps:** Discover thousands of AI-powered applications across many domains.
- **Diverse Modalities:** Work with text, image, video, audio,

# Hugging Face: The AI Community Building the Future

---

## About Hugging Face

Hugging Face is a vibrant AI community and the leading platform where machine learning developers, researchers, and organizations collaborate on models, datasets, and AI applications. The company fosters open collaboration to accelerate innovation across multiple AI modalities including text, image, video, audio, and even 3D.

With over **1 million models** and **250,000+ datasets** available, Hugging Face is the home base for machine learning enthusiasts and experts worldwide to create, share, and build cutting-edge AI solutions.

---

## Platform Highlights

- **Collaboration Hub:** Host and collaborate on unlimited public models, datasets, and applications.
- **Explore AI Apps:** Discover thousands of AI-powered applications across many domains.
- **Diverse Modalities:** Work with text, image, video, audio, and 3D data.
- **Build Your Portfolio:** Share your ML work publicly and showcase your expertise.
- **Accelerated ML Development:** Leverage the Hugging Face open source stack and advanced compute resources.

---

## Enterprise Solutions

Hugging Face offers tailored solutions for growing teams and enterprises looking to scale securely and efficiently.

**Key Enterprise Features:**

- **Advanced Security & Access Controls:** Single sign-on (SSO), audit logs, granular access with resource groups.
- **Private Storage & Dataset Viewers:** Securely manage and collaborate on private datasets with extra storage included per team member.
- **Analytics Dashboard:** Monitor repository usage and manage spending with comprehensive insights.
- **Flexible Compute Options:** Access boosted GPU quotas, ZeroGPU scalability, and premium inference providers.
- **Dedicated Support:** Priority customer support to maximize platform performance and uptime.
- **Billing Control:** Streamline your budgeting with managed billing and yearly commit plans.

Enterprise pricing is customizable based on contract needs; team plans start at $20 per user/month.

---

## Pricing Plans

- **PRO Individual Plan ($9/month):**
  - 10× private storage capacity
  - 20× inference credits
  - 8× ZeroGPU quota with highest priority
  - Space hosting and dev mode
  - Dataset viewer for private data
  - Pro profile badge and blog publishing

- **Team Plan ($20/user/month):**
  - Includes all PRO features plus:
  - Enterprise-grade security and access controls
  - Flexible billing and usage analytics
  - SSO integration and private storage boosts

Customized enterprise contracts are available for organizations requiring advanced security, scalability, and dedicated support.

---

## Community & Culture

Hugging Face is more than just a platform—it's a collaborative AI community passionate about open-source principles, inclusivity, and innovation. It empowers users worldwide to build, share, and grow together.

The company's culture emphasizes:

- **Openness:** Sharing knowledge and resources to help the AI field advance.
- **Collaboration:** Connecting developers, researchers, and enterprises to foster innovation.
- **Diversity of Thought:** Welcoming contributors from different backgrounds and specialties.
- **Continuous Learning:** Supporting education and growth via a rich ecosystem of tools, documentation, and community forums.

---

## Careers at Hugging Face

Hugging Face is continually looking for talented individuals passionate about AI, open source, and community-building. Careers focus on software engineering, research, platform development, enterprise solutions, and customer support.

Working at Hugging Face means joining a mission-driven startup with a commitment to transforming AI development globally, fostering a collaborative and inclusive workplace, and solving real-world challenges with cutting-edge technology.

---

## Why Choose Hugging Face?

- World's leading AI collaboration platform with a massive community
- Open-source-first approach enabling transparency and rapid innovation
- Extensive model and dataset libraries accessible to everyone
- Robust enterprise-grade features for scaling securely
- Flexible, affordable pricing suitable for individuals to large organizations
- Supportive, mission-driven company culture fostering growth and learning

---

**Join Hugging Face today and be part of the future of AI innovation!**

Visit: https://huggingface.co

----

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the vibrant and fast-growing AI community dedicated to building the future of machine learning. Serving as the central collaboration platform for ML enthusiasts and professionals alike, Hugging Face empowers engineers, scientists, and users worldwide to develop, share, and explore cutting-edge models, datasets, and applications together.

Their open and ethical approach to AI fosters a thriving ecosystem where innovation is democratized and accessible across all skill levels.

---

## What We Offer

- **Models:** Access and contribute to over 1 million machine learning models spanning various modalities including text, image, video, audio, and even 3D.

- **Datasets:** Browse and utilize from a repository of 250,000+ datasets for training, evaluation, and experimentation with real-world data from industry leaders like Nvidia, OpenAI, and Anthropic.

- **Spaces:** Deploy and share interactive AI applications easily within the community, with thousands of live projects demonstrating the possibilities of machine learning.

- **Community & Collaboration:** Our platform serves as a hub where users can host unlimited public projects, collaborate on shared goals, evaluate benchmarks, and create portfolios that showcase their AI expertise.

- **Compute & Enterprise Solutions:** For teams and organizations, Hugging Face offers scalable paid compute options and enterprise-grade tools to accelerate machine learning projects and deployment.

---

## Company Culture

Hugging Face champions an **open-source and open-community ethos**. With a strong focus on transparency, ethics, and inclusivity, Hugging Face encourages collaboration across borders and disciplines. The culture nurtures continuous learning, innovation, and mutual support within a welcoming environment for newcomers and experts alike.

Their commitment to building an "open and ethical AI future" highlights their dedication to responsible innovation, ensuring AI technology benefits the broader society.

---

## Customers and Community

Hugging Face serves a diverse array of users including:

- Independent AI researchers and developers building personal or community projects.
- Machine learning engineers and data scientists in academia and industry tackling real-world challenges.
- Enterprises leveraging AI models and data pipelines to power their products and services.
- Leading AI organizations and companies such as Microsoft, Nvidia, OpenAI, and Anthropic contribute and benefit from the platform.

The platform’s vibrant community provides a unique ecosystem fostering collaboration, knowledge sharing, and accelerated innovation.

---

## Careers & Opportunities

Joining Hugging Face means becoming part of a pioneering AI community and working alongside some of the brightest minds in machine learning. The company values:

- Passion for open-source software and AI ethics.
- Curiosity and eagerness to innovate.
- Collaborative spirit and a desire to contribute to shared success.
- Commitment to building scalable solutions that advance the ML field.

Whether you are a software engineer, research scientist, community manager, or AI enthusiast, Hugging Face offers exciting career paths to shape the future of AI.

---

## Get Involved

- **Explore** over a million models and hundreds of thousands of datasets.
- **Create** and share your own machine learning projects.
- **Join** an active, welcoming community dedicated to open AI innovation.
- **Accelerate** your projects with Hugging Face's enterprise and compute services.

Visit [huggingface.co](https://huggingface.co) to get started and be part of the AI community building the future.

---

## Visual Identity

- Signature colors: Yellow (#FFD21E), Orange (#FF9D00), Gray (#6B7280)
- Friendly and recognizable logo embodying collaboration and openness.

---

Hugging Face – The Home of Machine Learning Collaboration and Innovation.

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>